On local runtime bc RAM is better

DLC version: 2.3.5 is ok (for both local and google colab)

need to copy over:
- video folder analyzed pickles
  - must go into video folder and select all and then download zip. Otherwise will miss files.
  - will give multiple zips probably
  - double check that you have the correct number of files downloaded
- dlc models
- config.yaml

#troubleshooting

###File doesnt exist when it does error:

If when entering with vs code and see hidden .db suffixes-- can't use use_shelve=True during original analysis

#install other packages and setup

```
import os
import deeplabcut
import numpy as np
import pandas as pd
import pickle
from deeplabcut.utils import auxfun_multianimal, auxiliaryfunctions
import random
from pathlib import Path
import glob
```

# which model

```
project_path = "D:\\paulazhu\\demo-me-2021-07-14\\"
config_path = os.path.join(project_path, "config.yaml")
tracklets_config_path = os.path.join(project_path, "tracklets_config.yaml")
```

```
SHUFFLE = 1
TRACK_METHOD = "ellipse"  # Could also be "box", but "ellipse" was found to be more robust on this dataset.
```

```
from deeplabcut.utils.auxiliaryfunctions import edit_config
SNAPSHOT = 4 # not the actual number, but its index within the folder starting from 0
edit_config(config_path, {'snapshotindex': SNAPSHOT})
```

# Convert_detections2tracklets on local runtime

```
for video in [project_path + "videos\\221127_PZ89_1.avi"]:
#for video in glob.glob(project_path + "videos\\" + "*.avi"):
  deeplabcut.convert_detections2tracklets(
      config_path,
      [video],
      videotype='avi',
      shuffle=SHUFFLE,
      track_method=TRACK_METHOD,
      ignore_bodyparts=["tail1", "tail2", "tailend"],  # Some body parts can optionally be ignored during tracking for better assembly (but they are used later),
  )
```

# Stitch and filter tracklets

```
#for video in glob.glob(project_path + "videos\\" + "*.avi"):
for video in [project_path + "videos\\221127_PZ89_1.avi"]:
  deeplabcut.stitch_tracklets(
    config_path,
    [video],
    videotype='avi',
    shuffle=SHUFFLE,
    track_method=TRACK_METHOD,
    n_tracks=2, # if 2 doesn't work, try 3 or 1 depending on how clear 2 mice is; also seems to mess up create_labeled_video when not 3
  )
  #Filter the predictions to remove small jitter, if desired:
  deeplabcut.filterpredictions(config_path,
                                [video],
                                shuffle=SHUFFLE,
                                videotype='avi',
                                track_method = TRACK_METHOD)

```

# Make videos

```
from deeplabcut.utils.auxiliaryfunctions import edit_config
PCUTOFF = 0.1 # previously 0.01
edit_config(config_path, {'pcutoff': PCUTOFF})
```

# might not give any labels when n_tracks not 3 like in config.yaml
```
deeplabcut.create_labeled_video(
  config_path,
  [video],
  videotype='avi',
  shuffle=SHUFFLE,
  color_by="individual",
  keypoints_only=False,
  draw_skeleton=True,
  filtered=False,
  track_method=TRACK_METHOD,
)
```


# works better i think because different n_tracks
```
for video in glob.glob(project_path + "videos\\" + "*.avi"):
  deeplabcut.create_video_with_all_detections(
    config_path,
    [video],
    videotype='avi',
    shuffle=SHUFFLE,
    )
```